### 사업보고서 "재무제표 주석"에서 값 읽기 테스트

In [29]:
import requests
from bs4 import BeautifulSoup
import re
import pandas

import sys
module_path = "D:\PythonProject\data-gatherer\dart_fs_notes"
sys.path.append(module_path)
import mydart
import myutil

doc_url = "http://dart.fss.or.kr/report/viewer.do?rcpNo=20180402001524&dcmNo=6047053&eleId=16&offset=856727&length=413319&dtd=dart3.xsd"

val_a = val_b = 0.0
unit = None
# 재무제표 주석 문서에서 사용된 금액 단위 찾기 (확정급여채무)
# unit = mydart.find_unit(doc_url)
unit = None
response = requests.get(doc_url)
if response.status_code == 200:     # URL GET '200 정상'
    soup = BeautifulSoup(response.text, features="lxml")
    doc_text = soup.text
    for a in re.finditer(r"확정급여+[가-힣]+ 현재가치", doc_text):
    # for a in re.finditer(r"확정급여+[가-힣]+현재가치", doc_text):            
        sliced_text = doc_text[a.start()-100:a.start()]
        print(sliced_text)
        trimmed_text = sliced_text.replace(" ","")     # 문서에 있는 모든 공백 제거
        unit = mydart.get_unit(trimmed_text)
        if unit != None:
            break
    for a in re.finditer("퇴직급여채무", doc_text):
        sliced_text = doc_text[a.start()-100:a.start()]
        trimmed_text = sliced_text.replace(" ","")     # 문서에 있는 모든 공백 제거
        unit = mydart.get_unit(trimmed_text)
        if unit != None:
            break     
else:
    unit = None   


# 당기말의 "확정급여채무"와 "사외적립자산" 금액 찾기
# val_a, val_b = mydart.find_item(doc_url)
import pandas

val_a = val_b = 0.0                    
doc_data = pandas.read_html(doc_url)
for df in doc_data:
    row = df.shape[0]
    for i in range(0, row):
        item = str(df.iloc[i,0])
        if item.__contains__('확정급여채무의 현재가치') or item.__contains__('확정급여부채의 현재가치') or \
           item.__contains__('확정급여채무 현재가치') or item.__contains__('확정급여부채 현재가치') or \
           item.__contains__('퇴직급여채무'):
            if len(item) <= 30:
            # print(i, type(df.iloc[i,1]), df.iloc[i,1])
                str_a = str(df.iloc[i,1])
                if str_a == "-": val_a = 0.0
                else: val_a += mydart.str2num(str_a)       # 값을 합한다.
        elif item.__contains__('사외적립자산의 공정가치') or item.__contains__('사외적립자산 공정가치'):
            # print("B", item)              
            # print(i, type(df.iloc[i,1]), df.iloc[i,1])                           
            if len(item) <= 30:    
                str_b = str(df.iloc[i,1])
                if item == str_b: val_b = -1        # 값 찾기 오류
                elif str_b == "-": val_b = 0.0
                else: val_b = mydart.str2num(str_b)
        else:
            # print(i, "Nothing")
            continue    
    if (val_a != 0.0) and (val_b != 0.0):
        break      
    
print(unit, val_a, val_b, sep=' ')

 포함하는 경우를 제외하고는 비용으로 인식하고 있습니다. (나) 확정급여제도확정급여제도는 확정기여제도 이외의 모든 퇴직급여제도로서 순확정급여부채로 인식하는 금액은 보고기간말 현재 
결제하는 데 사용할 수 있는 사외적립자산의 보고기간말 현재 공정가치를 차감한 금액이며, 확정급여채무는 매년 독립적인 보험계리법인에 의해 예측단위적립방식에 따라 산정하고 있습니다. 
예상되는 장기유급휴가, 그 밖의 장기근속급여, 장기장애급여, 이익분배금과 상여금, 이연보상을 포함하고 있습니다. 기타장기종업원급여와 관련하여 부채로 인식할 금액은 보고기간말 현재 
 대한 혜택이 사용되기에 충분한 과세소득이 발생할 가능성이 높은 경우 최선의 추정치로 인식하고 있습니다. 이러한 추정치는 미래의 실제 법인세부담과 다를 수 있습니다. 확정급여채무 
장애율 및 조기퇴직률, 급여수령권을 갖는 피부양자가 있는 종업원의 비율, 의료급여제도의 경우 의료원가청구율, 할인율, 미래의 임금과 급여 수준 등이 포함되며, 이러한 가정에 따라 
. 퇴직급여부채(1) 퇴직급여부채의 내용은 다음과 같습니다.                                  (단위 : 천원)









구분
당기
전기





천원 4333527.0 211034.0
